In [1]:
import pandas as pd
import ast

# Cargar datos de usuarios
df_users = pd.read_csv("australian_user_items.csv")
df_users["items"] = df_users["items"].apply(ast.literal_eval)

# Cargar metadatos de juegos
df_meta = pd.read_csv("steam_juegos_metadata.csv").fillna("")
df_meta["appid"] = df_meta["appid"].astype(str)


In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

# Asegúrate de tener los stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

stopwords_en = set(stopwords.words("english"))

def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r"[^\w\s]", "", texto)
    tokens = word_tokenize(texto)
    return [t for t in tokens if t not in stopwords_en and len(t) > 2]

# --- 1. Corpus por usuario ---
corpus_usuarios = []
for row in df_users.itertuples():
    juegos = [j["item_id"] for j in row.items if "item_id" in j]
    if len(juegos) > 1:
        corpus_usuarios.append(juegos)

# --- 2. Corpus por juego con refuerzo de géneros y categorías ---
corpus_metadata = []

for row in df_meta.itertuples():
    tokens = [row.appid]  # El propio ID como token principal
    tokens += limpiar_texto(row.name)
    tokens += limpiar_texto(row.short_description)
    tokens += limpiar_texto(" ".join(row.developers))

    # Reforzar géneros (x3) y categorías (x2)
    if row.genres:
        tokens += limpiar_texto(row.genres) * 3
    if row.categories:
        tokens += limpiar_texto(row.categories) * 2

    corpus_metadata.append(tokens)

# --- 3. Combinar corpus ---
corpus_total = corpus_usuarios + corpus_metadata


[nltk_data] Downloading package punkt to /home/alvaro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/alvaro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from gensim.models import Word2Vec

modelo_w2v = Word2Vec(
    sentences=corpus_total,
    vector_size=100,     # tamaño del embedding
    window=5,            # tamaño de la ventana de contexto
    min_count=2,         # ignora tokens con menos de 2 apariciones
    sg=1,                # 1 = skip-gram (mejor para recomendaciones)
    workers=4,           # núcleos para paralelizar
    epochs=10            # número de épocas de entrenamiento
)

modelo_w2v.save("modelo_word2vec_mejorado.model")
print("✅ Modelo Word2Vec entrenado y guardado como 'modelo_word2vec_mejorado.model'")


✅ Modelo Word2Vec entrenado y guardado como 'modelo_word2vec_mejorado.model'


In [4]:
df_meta = pd.read_csv("steam_juegos_metadata.csv").fillna("")
id_to_name = dict(zip(df_meta["appid"].astype(str), df_meta["name"]))


In [ ]:
def recomendar_similares(juego_id, modelo, id_to_name, topn=20):
    if juego_id not in modelo.wv:
        print(f"⚠️ El juego {juego_id} no está en el modelo.")
        return []

    similares = modelo.wv.most_similar(juego_id, topn=100)
    recomendaciones = []

    for item_id, score in similares:
        if item_id in id_to_name:
            recomendaciones.append((item_id, id_to_name[item_id], round(score, 4)))
        if len(recomendaciones) >= topn:
            break

    return recomendaciones


In [ ]:
from gensim.models import Word2Vec

modelo = Word2Vec.load("modelo_word2vec_mejorado.model")
recomendaciones = recomendar_similares("298110", modelo, id_to_name)  # ARK: Survival Evolved

for item_id, nombre, score in recomendaciones:
    print(f"🎮 {item_id} - {nombre} (score: {score})")


🎮 298110 - Far Cry® 4 (score: 0.8657)
🎮 241930 - Middle-earth™: Shadow of Mordor™ (score: 0.7792)
🎮 209650 - Call of Duty®: Advanced Warfare - Gold Edition (score: 0.7662)
🎮 274940 - Depth (score: 0.7426)
🎮 209660 - Call of Duty®: Advanced Warfare - Gold Edition (score: 0.7422)
🎮 289690 - LARA CROFT AND THE TEMPLE OF OSIRIS™ (score: 0.7316)
🎮 302510 - Ryse: Son of Rome (score: 0.7283)
🎮 330840 - Game of Thrones - A Telltale Games Series (score: 0.7239)
🎮 313690 - LEGO® Batman™ 3: Beyond Gotham (score: 0.7114)
🎮 307690 - Sleeping Dogs: Definitive Edition (score: 0.7066)
